# Prompts in LangChain

## What are Prompts?

**Prompts** are the instructions you give to a language model to get desired outputs. In LangChain, prompts are **structured templates** that allow you to create dynamic, reusable instructions instead of hardcoding text.

### Why Use Prompt Templates?

1. **Reusability**: Write once, use with different inputs
2. **Consistency**: Maintain consistent formatting across your application
3. **Maintainability**: Update prompts in one place
4. **Validation**: LangChain validates that all required variables are provided
5. **Composability**: Combine prompts with models and parsers seamlessly

### Prompts in the LangChain Pipeline

```
Input Data → Prompt Template → Formatted Prompt → Model → Output
```

### Anatomy of a Good Prompt

1. **Instruction**: What you want the model to do
2. **Context**: Background information
3. **Input Data**: The specific data to process
4. **Output Format**: How you want the response structured

### Key Principle

**Better prompts = Better outputs**. Prompt engineering is the art of crafting effective instructions to get the best results from LLMs.

## PromptTemplate

### Definition

`PromptTemplate` is the simplest and most commonly used prompt class in LangChain. It creates a template with **placeholders** that get filled with actual values at runtime.

### Key Features

- **Variable Substitution**: Use `{variable_name}` as placeholders
- **Type Safety**: Validates that all required variables are provided
- **String-Based**: Works with simple text completion models
- **Composable**: Can be chained with models and parsers

### Use Cases

- Simple text generation tasks
- Single-turn completions
- Parameterized queries
- Content generation with templates

### Basic Syntax

```python
PromptTemplate(
    template="Your template with {variable}",
    input_variables=["variable"]
)
```

In [ ]:
# PromptTemplate Example
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv

# Load API keys
load_dotenv()

# Create model
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3
)
model = ChatHuggingFace(llm=llm)

# Create prompt template
prompt = PromptTemplate(
    template="Generate few points on {topic}.",
    input_variables=['topic']
)

# Chain prompt with model using LCEL
chain = prompt | model

# Invoke with actual value
result = chain.invoke({'topic': 'AI'})
print(result.content)

### Code Explanation:

1. **Import PromptTemplate**: From `langchain_core.prompts`
2. **Setup model**: Create a chat model (HuggingFace in this example)
3. **Create template**:
   - `template`: String with `{topic}` placeholder
   - `input_variables`: List of variable names (must match placeholders)
4. **Chain with LCEL**: `prompt | model` creates a pipeline
   - Data flows: input dict → prompt (formats) → model (generates)
5. **Invoke**: Pass dictionary with actual values
   - `{'topic': 'AI'}` → "Generate few points on AI."
6. **Get result**: `result.content` contains the generated text

### Advanced Template Examples:

```python
# Multiple variables
prompt = PromptTemplate(
    template="Write a {length} {style} story about {topic}",
    input_variables=["length", "style", "topic"]
)

# Multi-line template
prompt = PromptTemplate(
    template="""
    You are a {role}.
    Task: {task}
    Context: {context}
    
    Please provide a detailed response.
    """,
    input_variables=["role", "task", "context"]
)

# From template string (shorthand)
prompt = PromptTemplate.from_template(
    "Summarize this text: {text}"
)
```

### Best Practices:

✅ **DO:**
- Use descriptive variable names
- Include clear instructions
- Specify output format
- Test with different inputs

❌ **DON'T:**
- Use ambiguous placeholders
- Forget to list all variables
- Make prompts too complex
- Hardcode values that should be variables

## ChatPromptTemplate

### Definition

`ChatPromptTemplate` is designed for **conversational AI** applications. It structures prompts as a sequence of messages with different roles (System, Human, AI).

### Key Features

- **Role-Based Messages**: System, Human, AI messages
- **Conversation Structure**: Natural chat format
- **Memory Support**: Can include conversation history
- **System Instructions**: Persistent behavior guidelines

### Message Roles

| Role | Purpose | Example |
|------|---------|----------|
| **System** | Set AI behavior/persona | "You are a helpful assistant" |
| **Human** | User input | "What is the capital of France?" |
| **AI** | Model response | "The capital of France is Paris" |

### Use Cases

- Chatbots and assistants
- Multi-turn conversations
- Context-aware responses
- Customer support bots
- Interactive applications

### Why Use ChatPromptTemplate?

Modern chat models (GPT-4, Claude, Llama) are **instruction-tuned** and work better with structured messages than plain text.

In [ ]:
# ChatPromptTemplate with Memory Example
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv

load_dotenv()

# Create model
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3
)
model = ChatHuggingFace(llm=llm)

# Create chat prompt with system message and memory
prompt = ChatPromptTemplate([
    ('system', '''You are a Helpful AI assistant. Your role is to give short and 
     straight-to-point, direct, concise responses.
     Format: give your answer and then stop.'''),
    MessagesPlaceholder(variable_name='memory'),
    ('human', '{query}')
])

# Memory to store conversation history
memory = []

# Chat loop
while True:
    query = input('-->You: ')
    if query.lower() == 'exit':
        break
    
    # Create chain and invoke
    chain = prompt | model
    response = chain.invoke({'query': query, 'memory': memory})
    
    print('-->AI:', response.content)
    
    # Store in memory
    memory.append(HumanMessage(content=query))
    memory.append(AIMessage(content=response.content))

### Code Explanation:

1. **Import message classes**: `HumanMessage`, `AIMessage` for conversation history
2. **Create ChatPromptTemplate** with three parts:
   - **System message** `('system', '...')`: Sets AI behavior (persistent)
   - **MessagesPlaceholder** `variable_name='memory'`: Slot for conversation history
   - **Human message** `('human', '{query}')`: Current user input
3. **Memory list**: Stores conversation as message objects
4. **Chat loop**:
   - Get user input
   - Invoke chain with `query` and `memory`
   - Print response
   - Append both messages to memory
5. **Context awareness**: Model "remembers" previous exchanges via memory

### How Memory Works:

```python
# Turn 1
User: "My name is Alice"
AI: "Nice to meet you, Alice!"
memory = [HumanMessage("My name is Alice"), AIMessage("Nice to meet you, Alice!")]

# Turn 2
User: "What's my name?"
# Model sees: system message + memory + current query
AI: "Your name is Alice"
```

### MessagesPlaceholder Explained:

- **Purpose**: Insert a list of messages dynamically
- **Variable**: Must match the key in invoke dict
- **Flexibility**: Can be empty list or contain many messages

```python
# Empty memory (first turn)
chain.invoke({'query': 'Hello', 'memory': []})

# With history (later turns)
chain.invoke({'query': 'What did I ask?', 'memory': [HumanMessage(...), AIMessage(...)]})
```

### Alternative: Without Memory

```python
# Simple chat without memory
prompt = ChatPromptTemplate([
    ('system', 'You are a helpful assistant'),
    ('human', '{question}')
])

chain = prompt | model
response = chain.invoke({'question': 'What is AI?'})
```

## Message Types and Serialization

### Message Classes

LangChain provides specific classes for different message roles:

```python
from langchain_core.messages import (
    SystemMessage,    # System instructions
    HumanMessage,     # User input
    AIMessage,        # Model response
    FunctionMessage   # Function call results (for agents)
)
```

### Why Use Message Objects?

1. **Metadata Preservation**: Store additional info (timestamps, IDs)
2. **Type Safety**: Clear distinction between roles
3. **Compatibility**: Works with all chat models
4. **Serialization**: Easy to save/load conversations

### Creating Messages:

```python
# System message
sys_msg = SystemMessage(content="You are a helpful assistant")

# Human message
human_msg = HumanMessage(content="What is Python?")

# AI message
ai_msg = AIMessage(content="Python is a programming language")

# With metadata
msg = HumanMessage(
    content="Hello",
    additional_kwargs={"user_id": "123", "timestamp": "2024-02-04"}
)
```

In [ ]:
# Message Serialization Helper
from langchain_core.messages import HumanMessage, AIMessage

def messages_to_text(messages):
    """Convert message objects to readable text format."""
    lines = []
    for msg in messages:
        if isinstance(msg, HumanMessage):
            lines.append(f"Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            lines.append(f"AI: {msg.content}")
    return "\n".join(lines)

# Example usage
conversation = [
    HumanMessage(content="What is LangChain?"),
    AIMessage(content="LangChain is a framework for building LLM applications."),
    HumanMessage(content="What are its main components?"),
    AIMessage(content="Models, Prompts, Chains, Memory, and Agents.")
]

print(messages_to_text(conversation))

### Code Explanation:

1. **Purpose**: Convert message objects to plain text
2. **Use cases**:
   - Logging conversations
   - Saving to text files
   - Displaying chat history
   - Debugging
3. **Type checking**: `isinstance()` identifies message type
4. **Formatting**: Creates "Role: Content" format

### Saving/Loading Conversations:

```python
import json
from langchain_core.messages import messages_from_dict, messages_to_dict

# Save to JSON
messages_dict = messages_to_dict(conversation)
with open('chat_history.json', 'w') as f:
    json.dump(messages_dict, f)

# Load from JSON
with open('chat_history.json', 'r') as f:
    loaded_dict = json.load(f)
conversation = messages_from_dict(loaded_dict)
```

## Prompt Engineering Best Practices

### 1. Be Specific and Clear

❌ **Bad**: "Tell me about AI"

✅ **Good**: "Explain artificial intelligence in 3 bullet points for a beginner"

### 2. Provide Context

```python
prompt = PromptTemplate.from_template(
    """
    Context: You are a Python programming tutor.
    Student Level: {level}
    Question: {question}
    
    Provide a clear explanation with a code example.
    """
)
```

### 3. Specify Output Format

```python
prompt = PromptTemplate.from_template(
    """
    Analyze this text: {text}
    
    Provide your analysis in this format:
    - Main Topic:
    - Key Points:
    - Sentiment:
    """
)
```

### 4. Use Examples (Few-Shot Prompting)

```python
prompt = PromptTemplate.from_template(
    """
    Classify the sentiment of the following text.
    
    Examples:
    Text: "I love this product!"
    Sentiment: Positive
    
    Text: "This is terrible."
    Sentiment: Negative
    
    Text: "{text}"
    Sentiment:
    """
)
```

### 5. Control Output Length

```python
prompt = PromptTemplate.from_template(
    "Summarize this in exactly 2 sentences: {text}"
)
```

### 6. System Message Best Practices

```python
# Good system messages
system_messages = [
    "You are a helpful, concise assistant.",
    "You are an expert Python programmer. Provide code examples.",
    "You are a customer support agent. Be polite and professional.",
    "You answer in bullet points. Be brief and direct."
]
```

### 7. Temperature Settings

| Task | Temperature | Reason |
|------|-------------|--------|
| Factual Q&A | 0.0-0.3 | Deterministic, accurate |
| General Chat | 0.5-0.7 | Balanced |
| Creative Writing | 0.8-1.0 | Diverse, creative |
| Code Generation | 0.0-0.2 | Precise, correct |

### 8. Prompt Testing

```python
# Test with multiple inputs
test_cases = [
    {"topic": "AI"},
    {"topic": "Python"},
    {"topic": "LangChain"}
]

for test in test_cases:
    result = chain.invoke(test)
    print(f"Input: {test}")
    print(f"Output: {result.content}\n")
```

## Advanced Prompting Techniques

### 1. Chain-of-Thought Prompting

Encourage step-by-step reasoning:

```python
prompt = PromptTemplate.from_template(
    """
    Solve this problem step by step:
    {problem}
    
    Let's think through this:
    Step 1:
    """
)
```

### 2. Role Prompting

Assign a specific role:

```python
prompt = ChatPromptTemplate([
    ('system', 'You are a senior software architect with 20 years of experience.'),
    ('human', 'How should I design {feature}?')
])
```

### 3. Constraint-Based Prompting

Set clear boundaries:

```python
prompt = PromptTemplate.from_template(
    """
    Write a summary of {text}
    
    Constraints:
    - Maximum 50 words
    - Use simple language
    - Include only main points
    """
)
```

### 4. Multi-Step Prompting

Break complex tasks into steps:

```python
# Step 1: Generate ideas
ideas_prompt = PromptTemplate.from_template(
    "Generate 5 ideas for {topic}"
)

# Step 2: Evaluate ideas
eval_prompt = PromptTemplate.from_template(
    "Evaluate these ideas and pick the best one: {ideas}"
)

# Chain them
chain = ideas_prompt | model | eval_prompt | model
```

### 5. Conditional Prompting

Adapt based on input:

```python
def get_prompt(user_level):
    if user_level == "beginner":
        return PromptTemplate.from_template(
            "Explain {concept} in simple terms with examples"
        )
    else:
        return PromptTemplate.from_template(
            "Provide a technical explanation of {concept}"
        )
```

## Summary

### Prompt Types Covered

1. **PromptTemplate**: Simple string templates with variables
2. **ChatPromptTemplate**: Role-based messages for conversations
3. **MessagesPlaceholder**: Dynamic message insertion for memory
4. **Message Types**: HumanMessage, AIMessage, SystemMessage

### Key Takeaways

1. **Templates > Hardcoding**: Always use templates for reusability
2. **Chat Models Need ChatPromptTemplate**: Use role-based messages
3. **System Messages Matter**: Set behavior and constraints
4. **Memory = MessagesPlaceholder**: For conversation history
5. **LCEL Integration**: Prompts chain seamlessly with models
6. **Prompt Engineering**: Better prompts = better outputs

### Quick Decision Guide

```
Simple completion? → PromptTemplate
Chatbot/conversation? → ChatPromptTemplate
Need memory? → ChatPromptTemplate + MessagesPlaceholder
Complex task? → Multi-step prompting
Need consistency? → System message with constraints
```

### Prompt Engineering Checklist

✅ Clear instructions
✅ Relevant context
✅ Specified output format
✅ Examples (if needed)
✅ Appropriate temperature
✅ Tested with multiple inputs

### Next Steps in LangChain

After mastering prompts:
1. **Output Parsers**: Structure model outputs (JSON, Pydantic)
2. **Chains**: Combine prompts, models, and parsers
3. **Memory**: Advanced conversation management
4. **Agents**: Build autonomous systems
5. **RAG**: Retrieval-augmented generation

### Additional Resources

- [LangChain Prompts Documentation](https://python.langchain.com/docs/modules/model_io/prompts/)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [OpenAI Prompt Engineering](https://platform.openai.com/docs/guides/prompt-engineering)